- remove duration mean because additional features about time didn't seem to improve performance

In [1]:
import pandas as pd
import numpy as np
import warnings
import datetime
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, cohen_kappa_score
import lightgbm as lgb
import xgboost as xgb
from functools import partial
import json
import copy
import time
import seaborn as sns
import scipy as sp
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows",1000)
np.set_printoptions(precision=8)
warnings.filterwarnings("ignore")
import random

In [2]:
def qwk(a1, a2):
    max_rat = 3
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return np.round(1 - o / e, 8)

In [3]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])
        return -qwk(y, X_p)
    
    def fit(self, X, y, random_flg = False):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        if random_flg:
            initial_coef = [np.random.uniform(0.4,0.5), np.random.uniform(0.5,0.6), np.random.uniform(0.6,0.7)]
        else:
            initial_coef = [0.5, 1.5, 2.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        
    def predict(self, X, coef):
        return pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3])

    def coefficients(self):
        return self.coef_['x']

# install

In [4]:
%%time
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')
#specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
sample_submission = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

CPU times: user 1min 17s, sys: 11.2 s, total: 1min 29s
Wall time: 1min 29s


In [5]:
keep_id = train[train.type == "Assessment"][['installation_id']].drop_duplicates()
train = pd.merge(train, keep_id, on="installation_id", how="inner")
train = train[train.installation_id.isin(train_labels.installation_id.unique())]

In [6]:
assess_title = ['Mushroom Sorter (Assessment)', 'Bird Measurer (Assessment)',
       'Cauldron Filler (Assessment)', 'Cart Balancer (Assessment)', 'Chest Sorter (Assessment)']
def remove_index_calc(df):
    additional_remove_index = []
    for i, session in df.groupby('installation_id', sort=False):
        last_row = session.index[-1]
        session = session[session.title.isin(assess_title)]
        first_row = session.index[-1] + 1
        for j in range(first_row, last_row+1):
            additional_remove_index.append(j)                
    return additional_remove_index
additional_remove_index = remove_index_calc(train)
train = train[~train.index.isin(additional_remove_index)]

# Preprocess and Feature engineering

In [7]:
%%time
def encode_title(train, test):
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    list_of_title_eventcode = sorted(list(set(train['title_event_code'].unique()).union(set(test['title_event_code'].unique()))))

    list_of_user_activities = sorted(list(set(train['title'].unique()).union(set(test['title'].unique()))))
    list_of_event_code = sorted(list(set(train['event_code'].unique()).union(set(test['event_code'].unique()))))
    list_of_worlds = sorted(list(set(train['world'].unique()).union(set(test['world'].unique()))))
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = sorted(list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index))))

    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)

    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    train["misses"] = train["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    test["misses"] = test["event_data"].apply(lambda x: json.loads(x)["misses"] if "\"misses\"" in x else np.nan)
    
    train["level"] = train["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    test["level"] = test["event_data"].apply(lambda x: json.loads(x)["level"] if "\"level\"" in x else np.nan)
    
    train["round"] = train["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
    test["round"] = test["event_data"].apply(lambda x: json.loads(x)["round"] if "\"round\"" in x else np.nan)
    
    train["true"] = train["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)
    test["true"] = test["event_data"].apply(lambda x: 1 if "true" in x and "correct" in x else 0)

    train["false"] = train["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
    test["false"] = test["event_data"].apply(lambda x: 1 if "false" in x and "correct" in x else 0)
                
    return train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode

train, test, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, activities_world, list_of_title_eventcode = encode_title(train, test)

CPU times: user 1min 58s, sys: 4 s, total: 2min 2s
Wall time: 2min 1s


In [8]:
def get_data(user_sample, test_set=False):
    last_activity = 0
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    time_spent_each_act = {actv: 0 for actv in list_of_user_activities}
    title_eventcode_count = {str(ele): 0 for ele in list_of_title_eventcode}
    user_world_count = {"world_"+str(wor) : 0 for wor in activities_world.values()}
    
    last_session_time_sec = 0
    all_assessments = []
    accuracy_groups = {"0":0, "1":0, "2":0, "3":0}
    accumulated_accuracy_group = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0 
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    miss = 0
    crys_game_true = 0; crys_game_false = 0
    tree_game_true = 0; tree_game_false = 0
    magma_game_true = 0; magma_game_false = 0
    crys_game_acc = []; tree_game_acc = []; magma_game_acc = []
    
    for i, session in user_sample.groupby('game_session', sort=False):      
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
        session_world = session["world"].iloc[0]
        
        # get current session time in seconds
        if session_type != 'Assessment':
            time_spent = int(session['game_time'].iloc[-1] / 1000)
            time_spent_each_act[activities_labels[session_title]] += time_spent   
            
            if session_type == "Game":
                true = session['true'].sum()
                false = session['false'].sum() 
                if session_world == activities_world["CRYSTALCAVES"]:
                    crys_game_true += true
                    crys_game_false += false
                    crys_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                elif session_world == activities_world["TREETOPCITY"]:
                    tree_game_true += true
                    tree_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                elif session_world == activities_world["MAGMAPEAK"]:
                    magma_game_true += true
                    magma_game_false += false
                    tree_game_acc.append(true / (true + false) if (true + false) != 0 else 0)
                else:
                    pass
                
                
        if (session_type == 'Assessment') & (test_set or len(session)>1): # test set or session in train_label
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            true_attempts = all_attempts['event_data'].str.contains('true').sum() # true in target assess
            false_attempts = all_attempts['event_data'].str.contains('false').sum() # false in target assessment
            
            # from start of installation_id to the start of target assessment ------------------------
            features = user_activities_count.copy() # appearance of each type without duplicates
            features.update(time_spent_each_act.copy()) # cumulative gameplay time in each title
            features.update(title_eventcode_count.copy()) # apperance of combi of title and event_code
            features.update(user_world_count.copy()) # appearance of world with duplicates
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            features["misses"] = miss
            features['accumulated_actions'] = accumulated_actions
            
            if session_world == activities_world["CRYSTALCAVES"]:
                features["game_true"] = crys_game_true
                features["game_false"] = crys_game_false
                features['game_accuracy'] = crys_game_true / (crys_game_true + crys_game_false) if (crys_game_true + crys_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(crys_game_acc) if len(crys_game_acc) >=1 else 0
                features["last_game_acc"] = crys_game_acc[-1] if len(crys_game_acc) >=1 else 0
            elif session_world == activities_world["TREETOPCITY"]:
                features["game_true"] = tree_game_true
                features["game_false"] = tree_game_false
                features['game_accuracy'] = tree_game_true / (tree_game_true + tree_game_false) if (tree_game_true + tree_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(tree_game_acc) if len(tree_game_acc) >=1 else 0
                features["last_game_acc"] = tree_game_acc[-1] if len(tree_game_acc) >=1 else 0
            elif session_world == activities_world["MAGMAPEAK"]:
                features["game_true"] = magma_game_true
                features["game_false"] = magma_game_false
                features['game_accuracy'] = magma_game_true / (magma_game_true + magma_game_false) if (magma_game_true + magma_game_false) != 0 else 0
                features["game_accuracy_std"] = np.std(magma_game_acc) if len(magma_game_acc) >=1 else 0
                features["last_game_acc"] = magma_game_acc[-1] if len(magma_game_acc) >=1 else 0
            
            # unique type --------------------------------------------------------
            features['installation_id'] = session['installation_id'].iloc[-1]
            features['session_title'] = session_title
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[str(features['accuracy_group'])] += 1
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
                        
        n_of_title_eventcode = Counter(session['title_event_code']) 
        for key in n_of_title_eventcode.keys():
            title_eventcode_count[str(key)] += n_of_title_eventcode[key]
        miss += np.sum(session["misses"])
            
        user_world_count["world_"+str(session_world)] += session.shape[0]

        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type
    if test_set:
        return all_assessments[-1]
    return all_assessments

In [9]:
new_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort=False)), total=train.installation_id.nunique(), desc='Installation_id', position=0):
    new_train += get_data(user_sample)
new_train = pd.DataFrame(new_train)
print(new_train.shape)
del train


(17690, 466)


In [10]:
new_test = []
for ins_id, user_sample in tqdm(test.groupby('installation_id', sort=False), total=test.installation_id.nunique(), desc='Installation_id', position=0):
    a = get_data(user_sample, test_set=True)
    new_test.append(a)   
new_test = pd.DataFrame(new_test)
print(new_test.shape)
del test


(1000, 466)


# Feature selection

In [11]:
X_train = new_train.drop(['accuracy_group'],axis=1) 
lbl = preprocessing.LabelEncoder()
lbl.fit(list(X_train["installation_id"]))
X_train["installation_id"] = lbl.transform(list(X_train["installation_id"]))
remove_features = []
for i in X_train.columns:
    if X_train[i].std() == 0 and i not in remove_features:
        remove_features.append(i)
X_train = X_train.drop(remove_features, axis=1)
X_train = X_train[sorted(X_train.columns.tolist())]
y_train = new_train.accuracy_group
print(X_train.shape)
print(remove_features)

(17690, 440)
['12 Monkeys', 'Balancing Act', 'Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Costume Box', 'Crystal Caves - Level 1', 'Crystal Caves - Level 2', 'Crystal Caves - Level 3', 'Heavy, Heavier, Heaviest', 'Honey Cake', 'Lifting Heavy Things', 'Magma Peak - Level 1', 'Magma Peak - Level 2', 'Mushroom Sorter (Assessment)', 'Ordering Spheres', "Pirate's Tale", 'Rulers', 'Slop Problem', 'Treasure Map', 'Tree Top City - Level 1', 'Tree Top City - Level 2', 'Tree Top City - Level 3', 'Welcome to Lost Lagoon!']


In [12]:
X_test = new_test.drop(["installation_id", "accuracy_group"], axis=1)
X_test = X_test.drop(remove_features, axis=1)
X_test = X_test[sorted(X_test.columns.tolist())]

# modelling

In [13]:
def run_model(X_train, y_train, X_test):
    n_folds=5
    skf=GroupKFold(n_splits = n_folds)
    coefficients = []
    models = []
    train_qwk_scores = []
    test_qwk_scores = []
    train_qwk_scores_xgb = []
    test_qwk_scores_xgb = []

    lgbm_params = {
    'objective': 'regression','metric': 'rmse',"tree_learner": "serial", 
    "max_depth" : 5, "boosting": 'gbdt', "num_leaves" : 13, "learning_rate" : 0.01,
}
    xgb_params = {
    "objective" : "reg:squarederror","eval_metric" : "rmse","tree_learner": "serial",
    "max_depth" : 5,"boosting": 'gbdt',"num_leaves" : 13,"learning_rate" : 0.01,
}

    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train, X_train["installation_id"])):
        print("Fold "+str(i+1))
        optR = OptimizedRounder()
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]
        X_train2 = X_train2.drop(['installation_id'],axis=1)
    
        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        
        test2 = pd.concat([X_test2, y_test2], axis=1)
        second_last_index = list(set(test2.groupby('installation_id').tail(2).index)- set(test2.groupby('installation_id').tail(1).index))
        third_last_index = list(set(test2.groupby('installation_id').tail(3).index)- set(test2.groupby('installation_id').tail(2).index))
        test2 = test2.groupby('installation_id').tail(1)
        X_test2 = test2.drop(["accuracy_group", "installation_id"], axis=1)
        y_test2 = test2["accuracy_group"]
    
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
    
        clf = lgb.train(
        lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
        num_boost_round=10000,early_stopping_rounds=10,verbose_eval = 500
    )
        
        train_predict = clf.predict(X_train2, num_iteration = clf.best_iteration)
        test_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        
        models.append(clf)
        optR.fit(train_predict.reshape(-1,), y_train2)
        tmp_coefficients = optR.coefficients()
        opt_train_preds = optR.predict(train_predict.reshape(-1, ), tmp_coefficients)
        train_qwk_score = qwk(y_train2, opt_train_preds)
        opt_test_preds = optR.predict(test_predict.reshape(-1, ), tmp_coefficients)
        test_qwk_score = qwk(y_test2, opt_test_preds)
        train_qwk_scores.append(train_qwk_score)
        test_qwk_scores.append(test_qwk_score)
        coefficients.append(tmp_coefficients)
        
        #xgb_train = xgb.DMatrix(X_train2, label = y_train2)
        #xgb_eval = xgb.DMatrix(X_test2, label = y_test2)
        #watchlist = [(xgb_train, "train"), (xgb_eval, "eval")]
    
        #num_boost_round = 10000
        #clf_xgb = xgb.train(
        #xgb_params, xgb_train, num_boost_round, watchlist,
        #early_stopping_rounds=10, verbose_eval=500)
    
        #X_train2 = xgb.DMatrix(X_train2)
        #X_test2 = xgb.DMatrix(X_test2)
        #train_predict_xgb = clf_xgb.predict(X_train2, ntree_limit=clf_xgb.best_ntree_limit)
        #test_predict_xgb = clf_xgb.predict(X_test2, ntree_limit=clf_xgb.best_ntree_limit)
        
        #optR.fit(train_predict_xgb.reshape(-1,), y_train2)
        #tmp_coefficients_xgb = optR.coefficients()
        #opt_train_preds_xgb = optR.predict(train_predict_xgb.reshape(-1, ), tmp_coefficients_xgb)
        #train_qwk_score_xgb = qwk(y_train2, opt_train_preds_xgb)
        #opt_test_preds_xgb = optR.predict(test_predict_xgb.reshape(-1, ), tmp_coefficients_xgb)
        #test_qwk_score_xgb = qwk(y_test2, opt_test_preds_xgb)
        #train_qwk_scores_xgb.append(train_qwk_score_xgb)
        #test_qwk_scores_xgb.append(test_qwk_score_xgb)
    
    print("-----------------------------")
    print('train qwk list: ', train_qwk_scores, np.mean(train_qwk_scores))
    print('valid qwk list: ', test_qwk_scores, np.mean(test_qwk_scores))
    
    pred_value = np.zeros([X_test.shape[0]])
    avg_coefficients = np.mean(coefficients, axis=0)
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    
    return pred_value, np.mean(train_qwk_scores), np.mean(test_qwk_scores), avg_coefficients

In [14]:
pred_value = np.zeros([X_test.shape[0]])
tr_score = []; va_score = []; coefficients = []
num = 1
for i in range(num):
    pred, train_score, valid_score, coef = run_model(X_train, y_train, X_test)
    pred_value += pred / num
    tr_score.append(train_score)
    va_score.append(valid_score)
    coefficients.append(coef)
print(np.mean(tr_score), np.mean(va_score))

Fold 1
Training until validation scores don't improve for 10 rounds
[500]	training's rmse: 0.967055	valid_1's rmse: 1.0752
Early stopping, best iteration is:
[780]	training's rmse: 0.944465	valid_1's rmse: 1.06869
Fold 2
Training until validation scores don't improve for 10 rounds
[500]	training's rmse: 0.963292	valid_1's rmse: 1.03927
Early stopping, best iteration is:
[884]	training's rmse: 0.935236	valid_1's rmse: 1.03039
Fold 3
Training until validation scores don't improve for 10 rounds
[500]	training's rmse: 0.965216	valid_1's rmse: 1.00343
Early stopping, best iteration is:
[964]	training's rmse: 0.931254	valid_1's rmse: 0.991822
Fold 4
Training until validation scores don't improve for 10 rounds
[500]	training's rmse: 0.958057	valid_1's rmse: 1.09613
[1000]	training's rmse: 0.920554	valid_1's rmse: 1.08635
Early stopping, best iteration is:
[995]	training's rmse: 0.920855	valid_1's rmse: 1.08629
Fold 5
Training until validation scores don't improve for 10 rounds
[500]	training'

# prediction

In [15]:
test_coefficients = np.mean(coefficients, axis=0)
test_pred_class = pd.cut(pred_value, [-np.inf] + list(np.sort(test_coefficients)) + [np.inf], labels = [0, 1, 2, 3])
sample_submission["accuracy_group"] = test_pred_class.astype(int)
sample_submission.to_csv('submission.csv', index=False)

In [16]:
sample_submission["accuracy_group"].value_counts(normalize = True)

3    0.453
2    0.277
1    0.135
0    0.135
Name: accuracy_group, dtype: float64